# Pytorch Workflow

In [3]:
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

'2.6.0+cu124'

# Data Preparation
Data can be anything

In [4]:
# Create *known* parameters
weight = 0.7
bias = 0.3

#Create
start = 0
end =1
step = 0.02

X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10], len(X), len(y)

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]),
 50,
 50)

In [5]:
#Splitting data into training and testing
train_split = int(0.8*len(X))

X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]



# Model Building

In [6]:
from torch import nn
# Create linear regression model class
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1,
                                             requires_grad=True,
                                             dtype=torch.float))

    # Forward method to define the computation in the model
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.weights*x + self.bias

#Pytroch model building essentials

* torch.nn - all the blocks for building Neural networks (computational graphs)
* Parameters - what parameters should our model try and learn
* torch.nn.Module - the base class for NNs, if you subclass it, you should overwrite forward()
* torch.optim - optimizer helps with gradient descent


In [7]:
# Checking the contents of our pytorch model
torch.manual_seed(78)

lr = LinearRegressionModel()

list(lr.parameters())

[Parameter containing:
 tensor([-0.6238], requires_grad=True),
 Parameter containing:
 tensor([-1.2271], requires_grad=True)]

In [8]:
lr.state_dict()

OrderedDict([('weights', tensor([-0.6238])), ('bias', tensor([-1.2271]))])

In [9]:
weight, bias

(0.7, 0.3)

# Making predictions

In [10]:
with torch.inference_mode():
    y_preds = lr(X_test)
y_preds

tensor([[-1.7261],
        [-1.7386],
        [-1.7511],
        [-1.7635],
        [-1.7760],
        [-1.7885],
        [-1.8010],
        [-1.8134],
        [-1.8259],
        [-1.8384]])

In [11]:
#We could also do something similar with torch.no_grad()

with torch.no_grad():
    y_preds = lr(X_test)
y_preds

tensor([[-1.7261],
        [-1.7386],
        [-1.7511],
        [-1.7635],
        [-1.7760],
        [-1.7885],
        [-1.8010],
        [-1.8134],
        [-1.8259],
        [-1.8384]])

# Training Model

In [12]:
# Loss Functions
# Cost Function or criterion are popularly used as alias for loss functions

# We need Optimizer too, takes into account of the loss of the model and adjusts model performance

In [13]:
list(lr.parameters())

[Parameter containing:
 tensor([-0.6238], requires_grad=True),
 Parameter containing:
 tensor([-1.2271], requires_grad=True)]

In [14]:
lr.state_dict()

OrderedDict([('weights', tensor([-0.6238])), ('bias', tensor([-1.2271]))])

In [15]:
# L1 Loss - Mean Absolute Error
# MSE Loss - Mean Squared Error (L2 form)


In [16]:
# Setup a loss function
loss_fn = nn.L1Loss()


#Setup an optimizer
optimizer = torch.optim.SGD(params=lr.parameters(),
                            lr=0.01)



In [17]:
loss_fn

L1Loss()

## Building a training loop and testing loop
1. Loop Through data
2. Forward pass
3. calculate loss
4. optmizer zero grad
5. loss backward - move backwards through the netwrok to calculate the gradients of each of the parameters (back propagation)
6. optmizer step  (gradient descent)



In [18]:
torch.manual_seed(42)

epochs =300

epoch_count = []
loss_values = []
test_loss_values = []


for epoch in range(epochs):
    lr.train()
    #Forward pass
    y_pred = lr(X_train)
    #Loss calculation
    loss = loss_fn(y_pred, y_train)
    # print(loss)
    #Optimizer zero grad
    optimizer.zero_grad()
    #Perform Back propagation on the loss with respect
    loss.backward()
    #Step the optmizer
    optimizer.step()


    #Testing
    lr.eval() #turns off different settings which are not needed
    with torch.inference_mode(): #turns off gradient tracking and couple more
        #Do the forward pass
        test_pred = lr(X_test)

        #Calculate the loss
        test_loss = loss_fn(test_pred, y_test)

    if epoch%10==0:
        epoch_count.append(epoch)
        loss_values.append(loss)
        test_loss_values.append(loss)
        print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss} ")


Epoch: 0 | Loss: 2.0433435440063477 | Test Loss: 2.6917848587036133 
Epoch: 10 | Loss: 1.9281337261199951 | Test Loss: 2.557075023651123 
Epoch: 20 | Loss: 1.812923789024353 | Test Loss: 2.422365188598633 
Epoch: 30 | Loss: 1.69771409034729 | Test Loss: 2.2876553535461426 
Epoch: 40 | Loss: 1.5825040340423584 | Test Loss: 2.1529455184936523 
Epoch: 50 | Loss: 1.467294454574585 | Test Loss: 2.018235683441162 
Epoch: 60 | Loss: 1.3520845174789429 | Test Loss: 1.883526086807251 
Epoch: 70 | Loss: 1.2368746995925903 | Test Loss: 1.7488162517547607 
Epoch: 80 | Loss: 1.1216647624969482 | Test Loss: 1.6141064167022705 
Epoch: 90 | Loss: 1.0064548254013062 | Test Loss: 1.4793965816497803 
Epoch: 100 | Loss: 0.8912448883056641 | Test Loss: 1.344686508178711 
Epoch: 110 | Loss: 0.7760348916053772 | Test Loss: 1.209976315498352 
Epoch: 120 | Loss: 0.6608248353004456 | Test Loss: 1.0752663612365723 
Epoch: 130 | Loss: 0.5456148386001587 | Test Loss: 0.9405563473701477 
Epoch: 140 | Loss: 0.430404

In [19]:
lr.state_dict()

OrderedDict([('weights', tensor([0.3058])), ('bias', tensor([0.4654]))])

In [20]:
with torch.inference_mode():
    y_preds = lr(X_test)
print(y_preds)
print(y_test)

tensor([[0.7101],
        [0.7162],
        [0.7223],
        [0.7284],
        [0.7346],
        [0.7407],
        [0.7468],
        [0.7529],
        [0.7590],
        [0.7651]])
tensor([[0.8600],
        [0.8740],
        [0.8880],
        [0.9020],
        [0.9160],
        [0.9300],
        [0.9440],
        [0.9580],
        [0.9720],
        [0.9860]])


## Saving Model in pytorch

1. torch.save()
2. torch.load()
3. torch.nn.Module.load_state_dict()

In [23]:
from pathlib import Path

#Creating a directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

#Create model save path
MODEL_NAME = "LR_MODEL_TO_SAVE.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

#sAVING THE MODEL STATE DICT
torch.save(obj=lr.state_dict(),
           f=MODEL_SAVE_PATH)

In [24]:
## Loading pytorch model

lr.state_dict()

OrderedDict([('weights', tensor([0.3058])), ('bias', tensor([0.4654]))])

In [27]:
#To load a in a save dict we have to instantiate the model class

loaded_lr = LinearRegressionModel()
loaded_lr.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [28]:
loaded_lr.state_dict()

OrderedDict([('weights', tensor([0.3058])), ('bias', tensor([0.4654]))])

# Device agnostic code

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device", device)

Device cpu


In [32]:
class LinearRegressionModelv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(1,1)
    def forward(self, x):
        return self.l1(x)

torch.manual_seed(78)

model_1 = LinearRegressionModelv2()
model_1.state_dict()


OrderedDict([('l1.weight', tensor([[-0.3312]])),
             ('l1.bias', tensor([-0.6855]))])

In [37]:
X_train = X_train.to(device)
y_train = y_train.to(device)

X_test = X_test.to(device)
y_test = y_test.to(device)



In [33]:
model_1

LinearRegressionModelv2(
  (l1): Linear(in_features=1, out_features=1, bias=True)
)

In [34]:
model_1.to(device)

LinearRegressionModelv2(
  (l1): Linear(in_features=1, out_features=1, bias=True)
)